In [156]:
import pandas as pd
import numpy as np

In [144]:
all_cik_codes = pd.read_csv('cik_codes_with_feed_urls.csv', dtype='str')
all_cik_codes = all_cik_codes.drop('id', axis=1)
all_cik_codes.head()

,cik,companyname,entityid,primaryexchange,primarysymbol,siccode,sicdescription,feed_url,SD_url_2014,SD_url_2015,SD_url_2016,feed_retrieved_date
0,706688,AARON'S INC,2480,NYSE,AAN,7359,"Equipment Rental and Leasing, Not Elsewhere Cl...",https://www.sec.gov/cgi-bin/browse-edgar?owner...,http://www.sec.gov/Archives/edgar/data/706688/...,http://www.sec.gov/Archives/edgar/data/706688/...,NaN,2016-04-30
1,824142,AAON INC,938,Nasdaq Global Market,AAON,3585,Air-Conditioning and Warm Air Heating Equipmen...,https://www.sec.gov/cgi-bin/browse-edgar?owner...,http://www.sec.gov/Archives/edgar/data/824142/...,http://www.sec.gov/Archives/edgar/data/824142/...,NaN,2016-04-30
2,881890,ABAXIS INC,11929,Nasdaq Global Market,ABAX,3829,"Measuring and Controlling Devices, Not Elsewhe...",https://www.sec.gov/cgi-bin/browse-edgar?owner...,http://www.sec.gov/Archives/edgar/data/881890/...,http://www.sec.gov/Archives/edgar/data/881890/...,NaN,2016-04-30
3,1551152,ABBVIE INC.,883230,NYSE,ABBV,2834,Pharmaceutical Preparations,https://www.sec.gov/cgi-bin/browse-edgar?owner...,http://www.sec.gov/Archives/edgar/data/1551152...,http://www.sec.gov/Archives/edgar/data/1551152...,NaN,2016-04-30
4,1800,ABBOTT LABORATORIES,5054,NYSE,ABT,2834,Pharmaceutical Preparations,https://www.sec.gov/cgi-bin/browse-edgar?owner...,http://www.sec.gov/Archives/edgar/data/1800/00...,http://www.sec.gov/Archives/edgar/data/1800/00...,NaN,2016-04-30


In [171]:
import string
exclude = set(string.punctuation)
def clean_names(sub):
    sub['name_upper'] = sub['CompanyName'].str.upper()
    sub['name_clean'] = sub.name_upper.apply(lambda s: ''.join(ch for ch in s if ch not in exclude))
    return sub

def extract_cik_from_filing_url(url):
    try:
        return url.split('www.sec.gov/Archives/edgar/data/')[1].split('/')[0]
    except:
        print(url)


rsn_2015_scores = pd.read_csv('/Users/caged/Dropbox/RSN Mining/RSN 2015 Scores - MachineReadable.csv', dtype='str')
rsn_2015_scores = rsn_2015_scores.sort_values(by='CompanyName')
rsn_2015_scores = clean_names(rsn_2015_scores)
# I did some checking to make sure this was reasonable
rsn_2015_scores['extracted_cik_2015'] = rsn_2015_scores['2015_filing'].apply(extract_cik_from_filing_url)
print(len(rsn_2015_scores))
rsn_2015_scores.head()

155


,UID,CompanyName,MTD14_Pilot,Industry,Sector,Peer Group,2015_filing,2014_filing,MA1_1A_1,MA1_1A_2,...,MA5_19_3,MA5_19_4,MA5_19_5,MA5_19_2,MA5_19_1,MA5_20_1,MA5_20_2,name_upper,name_clean,extracted_cik_2015
95,116,3M Co.,1,Industrial Conglomerates,Industrials,Industrial Conglomerates,http://www.sec.gov/Archives/edgar/data/66740/0...,http://www.sec.gov/Archives/edgar/data/66740/0...,1,1,...,NaN,NaN,NaN,1,NaN,0,NaN,3M CO.,3M CO,66740
86,105,ABB LTD,1,Electrical Equipment,Industrials,Electrical Equipment,http://www.sec.gov/Archives/edgar/data/1091587...,http://www.sec.gov/Archives/edgar/data/1091587...,1,1,...,NaN,NaN,NaN,1,NaN,0,0,ABB LTD,ABB LTD,1091587
130,157,ASML Holding NV,NaN,Semiconductors and Semiconductor Equipment,Information Technology,Semiconductors and Semiconductor Equipment,http://www.sec.gov/Archives/edgar/data/937966/...,http://www.sec.gov/Archives/edgar/data/937966/...,1,1,...,NaN,NaN,NaN,1,1,0,NaN,ASML HOLDING NV,ASML HOLDING NV,937966
67,82,Abbott Laboratories,1,Healthcare Equipment and Supplies,Healthcare,Healthcare Equipment and Supplies,http://www.sec.gov/Archives/edgar/data/1800/00...,http://www.sec.gov/Archives/edgar/data/1800/00...,0,0,...,NaN,NaN,NaN,1,NaN,0,NaN,ABBOTT LABORATORIES,ABBOTT LABORATORIES,1800
93,112,"Acuity Brands, Inc.",NaN,Electrical Equipment,Industrials,Electrical Equipment,http://www.sec.gov/Archives/edgar/data/1144215...,http://www.sec.gov/Archives/edgar/data/1144215...,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACUITY BRANDS, INC.",ACUITY BRANDS INC,1144215


In [172]:
names_match = rsn_2015_scores.merge(all_cik_codes, left_on='extracted_cik_2015', right_on='cik', how='left')
print(len(names_match))
names_match.head()

155


,UID,CompanyName,MTD14_Pilot,Industry,Sector,Peer Group,2015_filing,2014_filing,MA1_1A_1,MA1_1A_2,...,entityid,primaryexchange,primarysymbol,siccode,sicdescription,feed_url,SD_url_2014,SD_url_2015,SD_url_2016,feed_retrieved_date
0,116,3M Co.,1,Industrial Conglomerates,Industrials,Industrial Conglomerates,http://www.sec.gov/Archives/edgar/data/66740/0...,http://www.sec.gov/Archives/edgar/data/66740/0...,1,1,...,5248,NYSE,MMM,3841,Surgical and Medical Instruments and Apparatus,https://www.sec.gov/cgi-bin/browse-edgar?owner...,http://www.sec.gov/Archives/edgar/data/66740/0...,http://www.sec.gov/Archives/edgar/data/66740/0...,NaN,2016-04-30
1,105,ABB LTD,1,Electrical Equipment,Industrials,Electrical Equipment,http://www.sec.gov/Archives/edgar/data/1091587...,http://www.sec.gov/Archives/edgar/data/1091587...,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,157,ASML Holding NV,NaN,Semiconductors and Semiconductor Equipment,Information Technology,Semiconductors and Semiconductor Equipment,http://www.sec.gov/Archives/edgar/data/937966/...,http://www.sec.gov/Archives/edgar/data/937966/...,1,1,...,10888,Nasdaq Global Market,ASML,3559,"Special Industry Machinery, Not Elsewhere Clas...",https://www.sec.gov/cgi-bin/browse-edgar?owner...,http://www.sec.gov/Archives/edgar/data/937966/...,http://www.sec.gov/Archives/edgar/data/937966/...,NaN,2016-04-30
3,82,Abbott Laboratories,1,Healthcare Equipment and Supplies,Healthcare,Healthcare Equipment and Supplies,http://www.sec.gov/Archives/edgar/data/1800/00...,http://www.sec.gov/Archives/edgar/data/1800/00...,0,0,...,5054,NYSE,ABT,2834,Pharmaceutical Preparations,https://www.sec.gov/cgi-bin/browse-edgar?owner...,http://www.sec.gov/Archives/edgar/data/1800/00...,http://www.sec.gov/Archives/edgar/data/1800/00...,NaN,2016-04-30
4,112,"Acuity Brands, Inc.",NaN,Electrical Equipment,Industrials,Electrical Equipment,http://www.sec.gov/Archives/edgar/data/1144215...,http://www.sec.gov/Archives/edgar/data/1144215...,0,0,...,88293,NYSE,AYI,3640,Electric Lighting And Wiring Equipment,https://www.sec.gov/cgi-bin/browse-edgar?owner...,http://www.sec.gov/Archives/edgar/data/1144215...,http://www.sec.gov/Archives/edgar/data/1144215...,NaN,2016-04-30


In [173]:
print('No simple match: ', sum(pd.isnull(names_match.siccode)))
print('Simple match: ', sum(~pd.isnull(names_match.siccode)))

No simple match:  29
Simple match:  126


### There are 29 companies that don't match


In [187]:
match = names_match[~pd.isnull(names_match.feed_url)]
no_match = names_match[pd.isnull(names_match.feed_url)]
no_match.head()

,UID,CompanyName,MTD14_Pilot,Industry,Sector,Peer Group,2015_filing,2014_filing,MA1_1A_1,MA1_1A_2,...,entityid,primaryexchange,primarysymbol,siccode,sicdescription,feed_url,SD_url_2014,SD_url_2015,SD_url_2016,feed_retrieved_date
1,105,ABB LTD,1,Electrical Equipment,Industrials,Electrical Equipment,http://www.sec.gov/Archives/edgar/data/1091587...,http://www.sec.gov/Archives/edgar/data/1091587...,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,79,BP plc,NaN,"Oil, Gas and Consumable Fuels",Energy,"Oil, Gas and Consumable Fuels",http://www.sec.gov/Archives/edgar/data/313807/...,http://www.sec.gov/Archives/edgar/data/313807/...,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,80,"Berkshire Hathaway, Inc.",NaN,Diversified Financial Services,Financials,Other Large Cap,http://www.sec.gov/Archives/edgar/data/1067983...,http://www.sec.gov/Archives/edgar/data/1067983...,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,159,Broadcom Corp.,NaN,Semiconductors and Semiconductor Equipment,Information Technology,Semiconductors and Semiconductor Equipment,http://www.sec.gov/Archives/edgar/data/1054374...,http://www.sec.gov/Archives/edgar/data/1054374...,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,168,"Canon, Inc.",NaN,"Technology Hardware, Storage and Peripherals",Information Technology,"Technology Hardware, Storage and Peripherals",http://www.sec.gov/Archives/edgar/data/16988/0...,http://www.sec.gov/Archives/edgar/data/16988/0...,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Now lets get the actual docs

In [188]:
disclosure_docs = pd.read_csv('disclosure_docs_list.csv')
disclosure_docs = disclosure_docs.drop('Unnamed: 0', axis=1)
disclosure_docs = disclosure_docs.rename(columns={'req_cik': 'req_company_name', 'req_company_name': 'req_cik'})
disclosure_docs.ret_filing_date = pd.to_datetime(disclosure_docs.ret_filing_date)
disclosure_docs.head()

,req_company_name,req_cik,req_url,ret_cik,ret_company_name,ret_description,ret_filing_date,ret_sec_accession_number,ret_title,ret_url
0,AARON'S INC,706688,http://www.sec.gov/Archives/edgar/data/706688/...,706688,AARON'S INC,formsdconflictminerals2014.htm,2014-05-28 12:50:08,0000706688-14-000029,SD,https://www.sec.gov/Archives/edgar/data/706688...
1,AARON'S INC,706688,http://www.sec.gov/Archives/edgar/data/706688/...,706688,AARON'S INC,0000706688-14-000029.txt,2014-05-28 12:50:08,0000706688-14-000029,Complete submission text file,https://www.sec.gov/Archives/edgar/data/706688...
2,AARON'S INC,706688,http://www.sec.gov/Archives/edgar/data/706688/...,706688,AARON'S INC,sdconflictminerals2014.htm,2015-05-22 12:06:05,0000706688-15-000148,SD_CONFLICT_MINERALS_2014,https://www.sec.gov/Archives/edgar/data/706688...
3,AARON'S INC,706688,http://www.sec.gov/Archives/edgar/data/706688/...,706688,AARON'S INC,0000706688-15-000148.txt,2015-05-22 12:06:05,0000706688-15-000148,Complete submission text file,https://www.sec.gov/Archives/edgar/data/706688...
4,AAON INC,824142,http://www.sec.gov/Archives/edgar/data/824142/...,824142,AAON INC,a2013formsd.htm,2014-06-02 13:49:18,0000824142-14-000080,AAON2013FORMSD,https://www.sec.gov/Archives/edgar/data/824142...


In [190]:
disclosure_docs_2015 = disclosure_docs[disclosure_docs.ret_filing_date.dt.year == 2015]
disclosure_docs_2015.head()

,req_company_name,req_cik,req_url,ret_cik,ret_company_name,ret_description,ret_filing_date,ret_sec_accession_number,ret_title,ret_url
2,AARON'S INC,706688,http://www.sec.gov/Archives/edgar/data/706688/...,706688,AARON'S INC,sdconflictminerals2014.htm,2015-05-22 12:06:05,0000706688-15-000148,SD_CONFLICT_MINERALS_2014,https://www.sec.gov/Archives/edgar/data/706688...
3,AARON'S INC,706688,http://www.sec.gov/Archives/edgar/data/706688/...,706688,AARON'S INC,0000706688-15-000148.txt,2015-05-22 12:06:05,0000706688-15-000148,Complete submission text file,https://www.sec.gov/Archives/edgar/data/706688...
7,AAON INC,824142,http://www.sec.gov/Archives/edgar/data/824142/...,824142,AAON INC,a2014formsd.htm,2015-05-29 11:36:18,0000824142-15-000071,SD,https://www.sec.gov/Archives/edgar/data/824142...
8,AAON INC,824142,http://www.sec.gov/Archives/edgar/data/824142/...,824142,AAON INC,exhibit101.htm,2015-05-29 11:36:18,0000824142-15-000071,EXHIBIT 1.01,https://www.sec.gov/Archives/edgar/data/824142...
9,AAON INC,824142,http://www.sec.gov/Archives/edgar/data/824142/...,824142,AAON INC,0000824142-15-000071.txt,2015-05-29 11:36:18,0000824142-15-000071,Complete submission text file,https://www.sec.gov/Archives/edgar/data/824142...


In [193]:
match.head()

,UID,CompanyName,MTD14_Pilot,Industry,Sector,Peer Group,2015_filing,2014_filing,MA1_1A_1,MA1_1A_2,...,entityid,primaryexchange,primarysymbol,siccode,sicdescription,feed_url,SD_url_2014,SD_url_2015,SD_url_2016,feed_retrieved_date
0,116,3M Co.,1,Industrial Conglomerates,Industrials,Industrial Conglomerates,http://www.sec.gov/Archives/edgar/data/66740/0...,http://www.sec.gov/Archives/edgar/data/66740/0...,1,1,...,5248,NYSE,MMM,3841,Surgical and Medical Instruments and Apparatus,https://www.sec.gov/cgi-bin/browse-edgar?owner...,http://www.sec.gov/Archives/edgar/data/66740/0...,http://www.sec.gov/Archives/edgar/data/66740/0...,NaN,2016-04-30
2,157,ASML Holding NV,NaN,Semiconductors and Semiconductor Equipment,Information Technology,Semiconductors and Semiconductor Equipment,http://www.sec.gov/Archives/edgar/data/937966/...,http://www.sec.gov/Archives/edgar/data/937966/...,1,1,...,10888,Nasdaq Global Market,ASML,3559,"Special Industry Machinery, Not Elsewhere Clas...",https://www.sec.gov/cgi-bin/browse-edgar?owner...,http://www.sec.gov/Archives/edgar/data/937966/...,http://www.sec.gov/Archives/edgar/data/937966/...,NaN,2016-04-30
3,82,Abbott Laboratories,1,Healthcare Equipment and Supplies,Healthcare,Healthcare Equipment and Supplies,http://www.sec.gov/Archives/edgar/data/1800/00...,http://www.sec.gov/Archives/edgar/data/1800/00...,0,0,...,5054,NYSE,ABT,2834,Pharmaceutical Preparations,https://www.sec.gov/cgi-bin/browse-edgar?owner...,http://www.sec.gov/Archives/edgar/data/1800/00...,http://www.sec.gov/Archives/edgar/data/1800/00...,NaN,2016-04-30
4,112,"Acuity Brands, Inc.",NaN,Electrical Equipment,Industrials,Electrical Equipment,http://www.sec.gov/Archives/edgar/data/1144215...,http://www.sec.gov/Archives/edgar/data/1144215...,0,0,...,88293,NYSE,AYI,3640,Electric Lighting And Wiring Equipment,https://www.sec.gov/cgi-bin/browse-edgar?owner...,http://www.sec.gov/Archives/edgar/data/1144215...,http://www.sec.gov/Archives/edgar/data/1144215...,NaN,2016-04-30
5,137,Alcatel-Lucent,NaN,Communications Equipment,Information Technology,Communications Equipment,http://www.sec.gov/Archives/edgar/data/886125/...,http://www.sec.gov/Archives/edgar/data/886125/...,1,1,...,10386,NYSE,ALU,3661,Telephone and Telegraph Apparatus,https://www.sec.gov/cgi-bin/browse-edgar?owner...,http://www.sec.gov/Archives/edgar/data/886125/...,http://www.sec.gov/Archives/edgar/data/886125/...,NaN,2016-04-30


In [200]:
from os import makedirs

In [204]:
for i, company in match.iterrows():
    dir_name = ' '.join([company.companyname.strip('\\'), '--', company.cik])
    makedirs(dir_name)

FileExistsError: [Errno 17] File exists: 'AMETEK INC/ -- 1037868'